In [2]:
import face_recognition
from PIL import Image, ImageDraw
import cv2
import numpy as np

In [3]:
image = face_recognition.load_image_file("image/001.jpg")
face_locations = face_recognition.face_locations(image)

In [4]:
scale_percent = 60 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv2.imshow("Resized image", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Resized Dimensions :  (2400, 3600, 3)


In [5]:
# face landmarks
face_landmarks_list = face_recognition.face_landmarks(resized)

print("I found{} face(s) in this photograph.".format(len(face_landmarks_list)))

#Create a PIL imagedraw object so we can draw on the picture
pil_image = Image.fromarray(resized)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:
    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))
        
    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=5)
        
#Show the picture
pil_image.show()

I found1 face(s) in this photograph.
The chin in this face has the following points: [(1375, 970), (1369, 1068), (1372, 1171), (1388, 1274), (1422, 1369), (1482, 1451), (1558, 1520), (1644, 1571), (1745, 1586), (1844, 1576), (1935, 1530), (2019, 1473), (2084, 1401), (2127, 1317), (2151, 1221), (2165, 1121), (2173, 1024)]
The left_eyebrow in this face has the following points: [(1438, 909), (1491, 848), (1573, 836), (1653, 860), (1729, 895)]
The right_eyebrow in this face has the following points: [(1819, 901), (1893, 873), (1974, 860), (2054, 879), (2100, 944)]
The nose_bridge in this face has the following points: [(1770, 967), (1764, 1029), (1759, 1088), (1753, 1151)]
The nose_tip in this face has the following points: [(1661, 1199), (1704, 1208), (1751, 1220), (1798, 1212), (1849, 1206)]
The left_eye in this face has the following points: [(1532, 961), (1575, 943), (1621, 948), (1663, 980), (1617, 979), (1569, 974)]
The right_eye in this face has the following points: [(1877, 994), 

In [6]:
#Face recognition
# To capture video from webcam. 
video_capture = cv2.VideoCapture(0)

# To use a video file as input 
#cap = cv2.VideoCapture(".../filename.mp4")

#Load a sample picture and learn how to recognize it.
T_image = face_recognition.load_image_file("image/001.jpg")
T_face_encoding = face_recognition.face_encodings(T_image)[0]

#Create arrays of known face encodings and their names
known_face_encodings = [
    T_face_encoding,
]

known_face_names = [
    "Tuan",
]

while True:
    #Grab a single frame of video
    ret,frame = video_capture.read()
    
    #Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:,:,::-1]
    
    #Find all the faces and face encodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    #Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        #See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        name = "Unknown"
        
        #If a match was found in known_face_encodings, just use the first one
        #if True in matches:
        #  first_match_index = matches.index(True)
        #  name = known_face_names[first_match_index]
        
        #Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
        #Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255),2)
        
        #Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0,0,255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom -6), font, 1.0, (255,255,255),1)
    
    #Display the resulting image
    cv2.imshow('video', frame)
    
    #Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()